In [1]:
import pandas as pd
import cv2
import os
import mediapipe as mp
import time
import matplotlib.pyplot as plt

In [2]:
import pickle
model_filename = 'Model/model.pkl'
knn = pickle.load(open(model_filename, 'rb'))

In [3]:
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(min_detection_confidence=0.6,
                    min_tracking_confidence=0.6,
                    static_image_mode=False,
                    smooth_landmarks=True,
                    model_complexity=2)

In [4]:
video_name = 'WPushUp2.mp4'
video_path = 'Videos/' + video_name

In [5]:
frame_count = 0
frame_numbers = []
data_dict = {}
desired_width = 480
desired_height = 360

In [6]:
cap = cv2.VideoCapture(video_path)

In [7]:
# Assuming df is your DataFrame
def calculate_angle(row):
    import math
    row = abs(row)

    # Accessing columns by their index positions (0 to 5) from the row
    radians = math.atan2(row[5] - row[3], row[4] - row[2]) - math.atan2(row[1] - row[3], row[0] - row[2])
    angle = math.degrees(radians)
    if angle < 0:
        angle += 360
    return angle

In [ ]:
while cap.isOpened():
    success, image = cap.read()
    if not success:
        break

    # Increment frame_count
    frame_count += 1
    print(f'Processing frame {frame_count}')

    # Resize the frame to the desired width and height
    image = cv2.resize(image, (desired_width, desired_height))

    # Convert the image from BGR to RGB
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    results = pose.process(image_rgb)

    if results.pose_landmarks:

        frame_numbers.append(frame_count)
        data_dict = {f"{landmark.name}_Point_x": [] for landmark in mp_pose.PoseLandmark}
        data_dict.update({f"{landmark.name}_Point_y": [] for landmark in mp_pose.PoseLandmark})
        data_dict.update({f"{landmark.name}_Point_z": [] for landmark in mp_pose.PoseLandmark})
        data_dict.update({f"{landmark.name}_visibility": [] for landmark in mp_pose.PoseLandmark})

        for landmark in mp_pose.PoseLandmark:
            x_list_name = f"{landmark.name}_Point_x"
            y_list_name = f"{landmark.name}_Point_y"
            z_list_name = f"{landmark.name}_Point_z"
            visibility_list_name = f"{landmark.name}_visibility"

            data_dict[x_list_name].append(results.pose_landmarks.landmark[landmark].x)
            data_dict[y_list_name].append(results.pose_landmarks.landmark[landmark].y)
            data_dict[z_list_name].append(results.pose_landmarks.landmark[landmark].z)
            data_dict[visibility_list_name].append(results.pose_landmarks.landmark[landmark].visibility)
                
        df = pd.DataFrame(data_dict)
        Angles_df = pd.DataFrame()
            
        ####################Angles
        ##########################################SHOULDER
    
        # LEFT_SHOULDER
        left_shoulder = df[[
            'LEFT_ELBOW_Point_x', 'LEFT_ELBOW_Point_y',
            'LEFT_SHOULDER_Point_x', 'LEFT_SHOULDER_Point_y',
            'LEFT_HIP_Point_x', 'LEFT_HIP_Point_y'
        ]]
        Angles_df['Left_Shoulder_Angle'] = left_shoulder.apply(calculate_angle, axis=1)
    
        # RIGHT_SHOULDER
        right_shoulder = df[[
            'RIGHT_ELBOW_Point_x', 'RIGHT_ELBOW_Point_y',
            'RIGHT_SHOULDER_Point_x', 'RIGHT_SHOULDER_Point_y',
            'RIGHT_HIP_Point_x', 'RIGHT_HIP_Point_y'
        ]]
        Angles_df['Right_Shoulder_Angle'] = right_shoulder.apply(calculate_angle, axis=1)
    
        ###########################################ELBOW
    
        # LEFT_ELBOW
        left_elbow = df[[
            'LEFT_WRIST_Point_x', 'LEFT_WRIST_Point_y',
            'LEFT_ELBOW_Point_x', 'LEFT_ELBOW_Point_y',
            'LEFT_SHOULDER_Point_x', 'LEFT_SHOULDER_Point_y'
        ]]
        Angles_df['Left_Elbow_Angle'] = left_elbow.apply(calculate_angle, axis=1)
    
        # RIGHT_ELBOW
        right_elbow = df[[
            'RIGHT_WRIST_Point_x', 'RIGHT_WRIST_Point_y',
            'RIGHT_ELBOW_Point_x', 'RIGHT_ELBOW_Point_y',
            'RIGHT_SHOULDER_Point_x', 'RIGHT_SHOULDER_Point_y'
        ]]
        Angles_df['Right_Elbow_Angle'] = right_elbow.apply(calculate_angle, axis=1)
    
        #############################################WRIST
    
        # LEFT_WRIST
        left_wrist = df[[
            'LEFT_ELBOW_Point_x', 'LEFT_ELBOW_Point_y',
            'LEFT_WRIST_Point_x', 'LEFT_WRIST_Point_y',
            'LEFT_SHOULDER_Point_x', 'LEFT_SHOULDER_Point_y'
        ]]
        Angles_df['Left_Wrist_Angle'] = left_wrist.apply(calculate_angle, axis=1)
    
        # RIGHT_WRIST
        right_wrist = df[[
            'RIGHT_ELBOW_Point_x', 'RIGHT_ELBOW_Point_y',
            'RIGHT_WRIST_Point_x', 'RIGHT_WRIST_Point_y',
            'RIGHT_SHOULDER_Point_x', 'RIGHT_SHOULDER_Point_y'
        ]]
        Angles_df['Right_Wrist_Angle'] = right_wrist.apply(calculate_angle, axis=1)
    
        ##############################################HIP
    
        # LEFT_HIP
        left_hip = df[[
            'LEFT_SHOULDER_Point_x', 'LEFT_SHOULDER_Point_y',
            'LEFT_HIP_Point_x', 'LEFT_HIP_Point_y',
            'LEFT_KNEE_Point_x', 'LEFT_KNEE_Point_y'
        ]]
        Angles_df['Left_Hip_Angle'] = left_hip.apply(calculate_angle, axis=1)
    
        # RIGHT_HIP
        right_hip = df[[
            'RIGHT_SHOULDER_Point_x', 'RIGHT_SHOULDER_Point_y',
            'RIGHT_HIP_Point_x', 'RIGHT_HIP_Point_y',
            'RIGHT_KNEE_Point_x', 'RIGHT_KNEE_Point_y'
        ]]
        Angles_df['Right_Hip_Angle'] = right_hip.apply(calculate_angle, axis=1)
    
        ##############################################KNEE
    
        # LEFT_KNEE
        left_knee = df[[
            'LEFT_HIP_Point_x', 'LEFT_HIP_Point_y',
            'LEFT_KNEE_Point_x', 'LEFT_KNEE_Point_y',
            'LEFT_ANKLE_Point_x', 'LEFT_ANKLE_Point_y'
        ]]
        Angles_df['Left_Knee_Angle'] = left_knee.apply(calculate_angle, axis=1)
    
        #RIGHT_KNEE
    
        Right_Knee = df[[
            'RIGHT_HIP_Point_x', 'RIGHT_HIP_Point_y',
            'RIGHT_KNEE_Point_x', 'RIGHT_KNEE_Point_y',
            'RIGHT_ANKLE_Point_x', 'RIGHT_ANKLE_Point_y'
        ]]
        Angles_df['Right_Knee_Angle'] = Right_Knee.apply(calculate_angle, axis=1)
    
        ###################################################NECK
        Neck = df[[
            'LEFT_SHOULDER_Point_x', 'LEFT_SHOULDER_Point_y',
            'NOSE_Point_x', 'NOSE_Point_y',
            'RIGHT_SHOULDER_Point_x', 'RIGHT_SHOULDER_Point_y'
        ]]
        Angles_df['Neck_Angle'] = Neck.apply(calculate_angle, axis=1)
    
        ###############################################ANKLE
    
        #LEFT_ANKLE
        Left_Ankle = df[[
            'LEFT_KNEE_Point_x', 'LEFT_KNEE_Point_y',
            'LEFT_ANKLE_Point_x', 'LEFT_ANKLE_Point_y',
            'LEFT_HEEL_Point_x', 'LEFT_HEEL_Point_y'
        ]]
        Angles_df['Left_Ankle_Angle'] = Left_Ankle.apply(calculate_angle, axis=1)
    
        #RIGHT_ANKLE
    
        Right_Ankle = df[[
            'RIGHT_KNEE_Point_x', 'RIGHT_KNEE_Point_y',
            'RIGHT_ANKLE_Point_x', 'RIGHT_ANKLE_Point_y',
            'RIGHT_HEEL_Point_x', 'RIGHT_HEEL_Point_y'
        ]]
        Angles_df['Right_Ankle_Angle'] = Right_Ankle.apply(calculate_angle, axis=1)
    
        ################################################HEEL
    
        #LEFT_HEEL
    
        Left_Heel = df[[
            'LEFT_ANKLE_Point_x', 'LEFT_ANKLE_Point_y',
            'LEFT_HEEL_Point_x', 'LEFT_HEEL_Point_y',
            'LEFT_FOOT_INDEX_Point_x', 'LEFT_FOOT_INDEX_Point_y'
        ]]
        Angles_df['Left_Heel_Angle'] = Left_Heel.apply(calculate_angle, axis=1)
    
        #RIGHT_HEEL
    
        Right_Heel = df[[
            'RIGHT_ANKLE_Point_x', 'RIGHT_ANKLE_Point_y',
            'RIGHT_HEEL_Point_x', 'RIGHT_HEEL_Point_y',
            'RIGHT_FOOT_INDEX_Point_x', 'RIGHT_FOOT_INDEX_Point_y'
        ]]
        Angles_df['Right_Heel_Angle'] = Right_Heel.apply(calculate_angle, axis=1)
    
        ################################################FOOT_INDEX
    
        #LEFT_FOOT_INDEX
    
        Left_Foot_Index = df[[
            'LEFT_ANKLE_Point_x', 'LEFT_ANKLE_Point_y',
            'LEFT_FOOT_INDEX_Point_x', 'LEFT_FOOT_INDEX_Point_y',
            'LEFT_HEEL_Point_x', 'LEFT_HEEL_Point_y'
        ]]
        Angles_df['Left_Foot_Index_Angle'] = Left_Foot_Index.apply(calculate_angle, axis=1)
    
        #RIGHT_FOOT_INDEX
        Right_Foot_Index = df[[
            'RIGHT_ANKLE_Point_x', 'RIGHT_ANKLE_Point_y',
            'RIGHT_FOOT_INDEX_Point_x', 'RIGHT_FOOT_INDEX_Point_y',
            'RIGHT_HEEL_Point_x', 'RIGHT_HEEL_Point_y'
        ]]
        Angles_df['Right_Foot_Index_Angle'] = Right_Foot_Index.apply(calculate_angle, axis=1)
        
        prediction = knn.predict(Angles_df)
        
        image_bgr = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)
        
        cv2.putText(image_bgr, f"Prediction: {prediction}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)

        cv2.imshow('MediaPipe Pose', image_bgr)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [11]:
cap.release()
cv2.destroyAllWindows()